# Accessibility Atlas -- Data Exploration

Disability demographics, web accessibility compliance, assistive technology usage, employment data, education statistics, and AAC resources.

**Datasets included:**
- US Census disability by county (3,222 counties, ACS 2022)
- US Census disability by state (52 states, ACS 2021)
- US Census national disability characteristics (S1810, 2022)
- US Census disability trends (2017-2022)
- US Census disability by race/ethnicity (9 groups, 2022)
- WHO Healthy Life Expectancy (185 countries, 2000-2021)
- Eurostat EU disability prevalence (36 countries)
- BLS disability employment (2024)
- IDEA special education (7.5M students, 51 states)
- WebAIM Million 2025 (top 1M website accessibility audit)
- WebAIM Screen Reader Survey #10 (1,539 respondents, 2024)
- ADA digital accessibility lawsuits (2017-2024)
- Section 508 federal compliance (245 agencies)
- AAC symbol library catalog (34 libraries)
- VizWiz VQA (4,319 blind user questions)
- WLASL sign language index (2,000 signs)

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 6)
matplotlib.rcParams['font.family'] = 'sans-serif'

## 1. US Disability by County (Census ACS 2022)

3,222 US counties with disability rates broken down by 6 types.

In [ ]:
counties = pd.read_csv("census_disability_by_county_2022.csv", dtype={"fips": str})
print(f"Counties: {len(counties):,}")
print(f"Total US population covered: {counties['total_population'].sum():,}")
print(f"Total with disability: {counties['disability_total'].sum():,}")
print(f"\nDisability rate range: {counties['disability_rate'].min():.1f}% - {counties['disability_rate'].max():.1f}%")
counties.head()

In [ ]:
# Top 15 counties by disability rate
top = counties.nlargest(15, "disability_rate")[["county_name", "total_population", "disability_rate"]]
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(range(len(top)), top["disability_rate"].values, color="#2563eb")
ax.set_yticks(range(len(top)))
ax.set_yticklabels(top["county_name"].values)
ax.set_xlabel("Disability Rate (%)")
ax.set_title("US Counties with Highest Disability Rates (ACS 2022)")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Disability type breakdown (national totals)
types = {
    "Hearing": counties["hearing_disability"].sum(),
    "Vision": counties["vision_disability"].sum(),
    "Cognitive": counties["cognitive_disability"].sum(),
    "Ambulatory": counties["ambulatory_disability"].sum(),
    "Self-care": counties["self_care_disability"].sum(),
    "Indep. Living": counties["independent_living_disability"].sum(),
}
fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(types.keys(), [v/1e6 for v in types.values()], color="#2563eb")
ax.set_ylabel("People (millions)")
ax.set_title("US Disability by Type (Census ACS 2022)")
for bar, val in zip(bars, types.values()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
            f"{val/1e6:.1f}M", ha='center', fontsize=10)
plt.tight_layout()
plt.show()

## 2. National Disability Characteristics (Census S1810, 2022)

All 6 disability types with breakdowns by sex, race/ethnicity, and age.

In [ ]:
with open("census_disability_characteristics_2022.json") as f:
    chars = json.load(f)

print(f"Overall: {chars['overall']['with_disability']:,} ({chars['overall']['rate_pct']}%)")
print(f"\nBy sex:")
for sex, data in chars['by_sex'].items():
    print(f"  {sex}: {data['rate_pct']}%")
print(f"\nBy race/ethnicity:")
for race, data in chars['by_race_ethnicity'].items():
    print(f"  {race}: {data['rate_pct']}% ({data['with_disability']:,})")

# Plot by race
fig, ax = plt.subplots(figsize=(12, 5))
races = list(chars['by_race_ethnicity'].keys())
rates = [chars['by_race_ethnicity'][r]['rate_pct'] for r in races]
labels = [r.replace('_', ' ').title() for r in races]
bars = ax.barh(range(len(races)), rates, color="#2563eb")
ax.set_yticks(range(len(races)))
ax.set_yticklabels(labels)
ax.set_xlabel("Disability Rate (%)")
ax.set_title("US Disability Rate by Race/Ethnicity (2022)")
ax.axvline(x=13.4, color="#dc2626", linestyle="--", label="National avg (13.4%)")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Disability rate by age group
fig, ax = plt.subplots(figsize=(10, 5))
ages = list(chars['by_age'].keys())
rates = [chars['by_age'][a]['rate_pct'] for a in ages]
labels = [a.replace('_', ' ').replace('to', '-') for a in ages]
ax.bar(labels, rates, color="#059669")
ax.set_ylabel("Disability Rate (%)")
ax.set_title("US Disability Rate by Age Group (2022)")
for i, (r, l) in enumerate(zip(rates, labels)):
    ax.text(i, r + 1, f"{r}%", ha='center', fontsize=10)
plt.tight_layout()
plt.show()

## 3. US Disability Trends (2017-2022)

In [ ]:
with open("census_disability_national_trends.json") as f:
    trends = json.load(f)
years = [d["year"] for d in trends["data"]]
rates = [d["disability_rate_pct"] for d in trends["data"]]
counts = [d["with_disability"] for d in trends["data"]]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(years, rates, 'o-', color="#2563eb", linewidth=2, markersize=8)
ax1.set_xlabel("Year")
ax1.set_ylabel("Disability Rate (%)")
ax1.set_title("US Disability Prevalence Trend")
ax1.annotate("COVID-19\n(2020 data\nnot released)", xy=(2020, 13.0), fontsize=9, color="gray", ha="center")

ax2.bar(years, [c/1e6 for c in counts], color="#059669")
ax2.set_xlabel("Year")
ax2.set_ylabel("People with Disability (millions)")
ax2.set_title("US Population with Disability")
plt.tight_layout()
plt.show()

## 4. BLS Disability Employment (2024)

Employment, labor force participation, and unemployment rates by disability status.

In [ ]:
with open("bls_disability_employment_2024.json") as f:
    bls = json.load(f)

stats = bls["overall_statistics"]
print("Employment Statistics (2024):")
for group in ["with_disability", "without_disability"]:
    s = stats[group]
    label = "With disability" if "with" in group else "Without disability"
    print(f"  {label}:")
    print(f"    Labor force participation: {s.get('labor_force_participation_rate', 'N/A')}%")
    print(f"    Employment-population ratio: {s.get('employment_population_ratio', 'N/A')}%")
    print(f"    Unemployment rate: {s.get('unemployment_rate', 'N/A')}%")

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 5))
metrics = ["Labor Force\nParticipation", "Employment-\nPopulation Ratio", "Unemployment\nRate"]
dis_vals = [stats["with_disability"].get("labor_force_participation_rate", 0),
            stats["with_disability"].get("employment_population_ratio", 0),
            stats["with_disability"].get("unemployment_rate", 0)]
no_dis_vals = [stats["without_disability"].get("labor_force_participation_rate", 0),
               stats["without_disability"].get("employment_population_ratio", 0),
               stats["without_disability"].get("unemployment_rate", 0)]
x = range(len(metrics))
w = 0.35
ax.bar([i - w/2 for i in x], dis_vals, w, label="With disability", color="#dc2626")
ax.bar([i + w/2 for i in x], no_dis_vals, w, label="Without disability", color="#2563eb")
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.set_ylabel("Rate (%)")
ax.set_title("Disability Employment Gap (BLS 2024)")
ax.legend()
plt.tight_layout()
plt.show()

## 5. IDEA Special Education (2022-23)

7.5 million students served under the Individuals with Disabilities Education Act.

In [ ]:
with open("idea_special_education_2023.json") as f:
    idea = json.load(f)

print(f"Total students: {idea['national_summary']['total_students_served']:,}")
print(f"\nDisability categories:")
for cat in idea["disability_categories"]:
    print(f"  {cat['category']}: {cat['count']:,} ({cat['percentage']}%)")

# Plot categories
cats = idea["disability_categories"][:8]  # Top 8
fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(range(len(cats)), [c["count"] for c in reversed(cats)], color="#6366f1")
ax.set_yticks(range(len(cats)))
ax.set_yticklabels([c["category"][:35] for c in reversed(cats)])
ax.set_xlabel("Students Served")
ax.set_title("IDEA Special Education by Disability Category (2022-23)")
plt.tight_layout()
plt.show()

## 6. WHO Healthy Life Expectancy (2000-2021)

Years of healthy life (HALE) for 185 countries across 22 years.

In [ ]:
hale = pd.read_csv("who_hale_long.csv")
print(f"Records: {len(hale):,}")
print(f"Countries: {hale['country_code'].nunique()}")
print(f"Years: {hale['year'].min()}-{hale['year'].max()}")
print(f"\nHALE range: {hale['hale_years'].min():.1f} - {hale['hale_years'].max():.1f} years")

region_avg = hale.groupby(["region", "year"])["hale_years"].mean().reset_index()
fig, ax = plt.subplots(figsize=(12, 6))
for region in sorted(region_avg["region"].unique()):
    subset = region_avg[region_avg["region"] == region]
    ax.plot(subset["year"], subset["hale_years"], label=region, linewidth=2)
ax.set_xlabel("Year")
ax.set_ylabel("Healthy Life Expectancy (years)")
ax.set_title("WHO Healthy Life Expectancy by Region (2000-2021)")
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
plt.tight_layout()
plt.show()

## 7. Eurostat EU Disability Prevalence

36 EU/EEA countries with disability prevalence rates and employment gaps.

In [ ]:
with open("eurostat_disability_eu.json") as f:
    eu = json.load(f)

print(f"Countries: {len(eu['countries'])}")
print(f"EU27 summary: {eu['eu27_summary_2023']}")

# Plot top countries by disability rate
countries_data = eu["countries"]
if isinstance(countries_data, list):
    sorted_c = sorted(countries_data, key=lambda x: x.get("total_disability_rate_pct", 0) or 0, reverse=True)
    fig, ax = plt.subplots(figsize=(14, 8))
    names = [c.get("country", c.get("name", "?"))[:20] for c in sorted_c[:25]]
    rates = [c.get("total_disability_rate_pct", 0) or 0 for c in sorted_c[:25]]
    ax.barh(range(len(names)), rates, color="#059669")
    ax.set_yticks(range(len(names)))
    ax.set_yticklabels(names)
    ax.set_xlabel("Disability Rate (%)")
    ax.set_title("EU/EEA Disability Prevalence by Country (Eurostat GALI)")
    ax.axvline(x=eu['eu27_summary_2023'].get('total_disability_rate_pct', 26.9), 
               color="#dc2626", linestyle="--", label="EU27 avg")
    ax.legend()
    plt.tight_layout()
    plt.show()

## 8. WebAIM Million 2025

Automated WCAG accessibility analysis of the top 1,000,000 website home pages.

In [ ]:
with open("webaim_million_2025.json") as f:
    webaim = json.load(f)

print(f"Pages tested: {webaim['summary']['total_pages_tested']:,}")
print(f"Total errors: {webaim['summary']['total_errors_detected']:,}")
print(f"Avg errors/page: {webaim['summary']['average_errors_per_page']}")
print(f"Pages with failures: {webaim['summary']['pages_with_wcag_failures_pct']}%")
print("\nMost common WCAG failures:")
for f_item in webaim["common_failures"]:
    print(f"  {f_item['type']:30s} {f_item['pages_affected_pct']}% of pages")

trends = webaim["yearly_trends"]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(trends["years"], trends["pages_with_failures_pct"], 'o-', color="#dc2626", linewidth=2)
ax1.set_xlabel("Year")
ax1.set_ylabel("Pages with Failures (%)")
ax1.set_title("WCAG Failure Rate Over Time")
ax1.set_ylim(90, 100)

ax2.plot(trends["years"], trends["low_contrast_pct"], 'o-', label="Low Contrast", linewidth=2)
ax2.plot(trends["years"], trends["missing_alt_text_pct"], 's-', label="Missing Alt Text", linewidth=2)
ax2.plot(trends["years"], trends["empty_links_pct"], '^-', label="Empty Links", linewidth=2)
ax2.set_xlabel("Year")
ax2.set_ylabel("Pages Affected (%)")
ax2.set_title("Top WCAG Failures Over Time")
ax2.legend()
plt.tight_layout()
plt.show()

## 9. Screen Reader Users & AAC Resources

In [ ]:
with open("webaim_screen_reader_survey_2024.json") as f:
    survey = json.load(f)

sr = survey["primary_screen_reader"]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
labels = [s["name"] for s in sr if s["pct"] >= 2]
sizes = [s["pct"] for s in sr if s["pct"] >= 2]
colors = ["#2563eb", "#059669", "#6366f1", "#d97706", "#dc2626", "#8b5cf6"]
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors[:len(labels)])
ax1.set_title("Primary Screen Reader (Desktop)")

# AAC symbol library comparison
with open("aac_vocabulary_data.json") as f:
    aac = json.load(f)
libs = aac["symbol_libraries"]["libraries"]
lib_names = [l["name"][:15] for l in libs if l["total_symbols"] and l["total_symbols"] > 100]
lib_counts = [l["total_symbols"] for l in libs if l["total_symbols"] and l["total_symbols"] > 100]
ax2.barh(range(len(lib_names)), lib_counts, color="#d97706")
ax2.set_yticks(range(len(lib_names)))
ax2.set_yticklabels(lib_names)
ax2.set_xlabel("Number of Symbols")
ax2.set_title("AAC Symbol Libraries (Open-Licensed)")
plt.tight_layout()
plt.show()

## 10. ADA Lawsuits & VizWiz

In [ ]:
with open("ada_digital_lawsuits.json") as f:
    lawsuits = json.load(f)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.bar(lawsuits["years"], lawsuits["total_lawsuits"], color="#dc2626")
ax1.set_xlabel("Year")
ax1.set_ylabel("Number of Lawsuits")
ax1.set_title("ADA Digital Accessibility Lawsuits (2017-2024)")

vizwiz = pd.read_csv("vizwiz_val_annotations.csv")
answer_types = vizwiz["answer_type"].value_counts()
ax2.bar(answer_types.index, answer_types.values, color="#6366f1")
ax2.set_xlabel("Answer Type")
ax2.set_ylabel("Count")
ax2.set_title(f"VizWiz: Questions from Blind Users ({len(vizwiz):,} total)")
plt.tight_layout()
plt.show()
print(f"VizWiz answerable: {vizwiz['answerable'].mean()*100:.1f}%")

---

**Author:** Luke Steuber | [lukesteuber.com](https://lukesteuber.com) | [@lukesteuber.com](https://bsky.app/profile/lukesteuber.com)

**Source:** [Data Trove](https://dr.eamer.dev/datavis/data_trove/) at dr.eamer.dev